In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import layers
from keras.datasets import mnist, fashion_mnist, cifar10
from IPython.display import SVG
from keras.utils import model_to_dot

In [ ]:
def plot_image(image, cmap="binary"):
    plt.imshow(image, cmap=cmap)
    plt.axis("off")

def show_model(model):
    if not COLAB:
        display(SVG(model_to_dot(model,show_shapes=True).create(prog='dot', format='svg')))
    else:
        display(SVG(model_to_dot(model, show_shapes=True, dpi=72).create(prog='dot', format='svg')))

In [ ]:
#(X_tr, y_tr), (X_te, y_te) = fashion_mnist.load_data()
(X_tr, y_tr), (X_te, y_te) = mnist.load_data()

In [ ]:
X_tr.shape, X_te.shape

In [ ]:
np.unique(y_tr), np.unique(y_te)

In [ ]:
plt.figure(figsize=(12,12))
cont = 1
for clase in range(10):
  inds = np.where(y_tr==clase)[0]
  for ind in inds[:10]:
    plt.subplot(10,10,cont)
    plot_image(X_tr[ind])
    plt.title(str(y_tr[ind]))
    cont = cont + 1

In [ ]:
X_tr2 = X_tr[(y_tr == 5) | (y_tr == 9)] # clases 5 y 9 son clases ok
X_te2 = X_te[(y_te == 5) | (y_te == 9)| (y_te == 6)] # clases 5 y 9 son ok, clase 6 es ko
y_te2 = y_te[(y_te == 5) | (y_te == 9)| (y_te == 6)]
y_te2 = 1*(y_te2 == 6)

In [ ]:
np.mean(y_te2)

In [ ]:
from sklearn.model_selection import train_test_split
X_tr2, X_val2 = train_test_split(X_tr2, test_size=0.3, random_state=1) # el 30% lo llevo a validación

In [ ]:
from tensorflow import keras

In [ ]:
X_tr2.shape

In [ ]:
X_tr2.min(), X_tr2.max(), X_val2.min(), X_val2.max(), X_te2.min(), X_te2.max()

In [ ]:
X_tr2 = X_tr2 / 255
X_val2 = X_val2 / 255
X_te2 = X_te2 / 255

In [ ]:
X_tr2.min(), X_tr2.max(), X_val2.min(), X_val2.max(), X_te2.min(), X_te2.max()

In [ ]:
# defino mi red como una lista de capas:

model = keras.Sequential(
    [
     keras.Input((28,28)),
     keras.layers.Flatten(),
     keras.layers.Dense(100, activation="relu"),
     keras.layers.Dense(5, activation="relu"),
     keras.layers.Dense(100, activation="relu"),
     keras.layers.Dense(28*28, activation="sigmoid"),
     keras.layers.Reshape((28,28))
    ]
)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='rmsprop', loss="mse")

In [ ]:
from matplotlib.ticker import MaxNLocator

def plot_history(historia):
    f = plt.figure(figsize=(4,4))
    h = historia.history
    aux = range(1,len(h["loss"])+1)
    mejor_epoca = np.argmin(h["val_loss"])
    plt.plot(aux, h["loss"], label="entrenamiento")
    plt.plot(aux, h["val_loss"], label="validación")
    plt.plot(mejor_epoca+1, h["val_loss"][mejor_epoca], 'or')
    plt.title('Loss', fontsize=18)
    plt.xlabel('Época', fontsize=18)
    plt.xticks(fontsize=12); plt.yticks(fontsize=12)
    plt.legend()
    f.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

In [ ]:
lista_callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="best_model.keras",
        monitor="val_loss",
        #save_best_only=True,
    )
]

In [ ]:
historia = model.fit(X_tr2, X_tr2, batch_size=128, epochs=200, callbacks=lista_callbacks,
                     validation_data=(X_val2, X_val2))

In [ ]:
plot_history(historia)

In [ ]:
# cargo best_model de fichero, que es el punto rojo (donde mejor métrica en validación)
model = keras.models.load_model("best_model.keras")

In [ ]:
errores_signo = (X_te2 - model.predict(X_te2))
errores_totales = errores_signo**2

errores_totales.shape

In [ ]:
error_por_imagen = (errores_totales.sum(axis=1).sum(axis=1)/(28*28))
error_por_imagen.shape

In [ ]:
inds = np.argsort(error_por_imagen)[::-1] # saco los índices de los errores de mayor a menor magnitud

In [ ]:
y_te2[inds[:100]]

In [ ]:
len(y_te2)

In [ ]:
# 10 imágenes de tipo 0 (ok) en training:

plt.figure(figsize=(12,12))
cont = 1
for clase in range(1):
  for ind in range(10,20):
    plt.subplot(10,10,cont)
    plot_image(X_tr2[ind])
    cont = cont + 1

In [ ]:
# 10 imágenes de tipo 0 (ok) en test y 10 de tipo 1 (ko) en test:


plt.figure(figsize=(12,12))
cont = 1
for clase in range(2):
  inds = np.where(y_te2==clase)[0]
  for ind in inds[20:30]:
    plt.subplot(10,10,cont)
    plot_image(X_te2[ind])
    plt.title(str(y_te2[ind]))
    cont = cont + 1

In [ ]:
# 10 imágenes más anómalas en test:

plt.figure(figsize=(12,12))
cont = 1
for ind in inds[100:110]:
  plt.subplot(10,10,cont)
  plot_image(X_te2[ind])
  cont = cont + 1

In [ ]:
def show_error(error):
    img = np.zeros((error.shape[0],error.shape[1],3))
    img[:,:,1] = np.clip(error, 0, 1)
    img[:,:,2] = np.clip(error, 0, 1)

    img[:,:,0] = -np.clip(error, -1, 0)
    img[:,:,1] = img[:,:,1] - np.clip(error, -1, 0)

    plt.imshow(1-img, vmin=0, vmax=1)
    plt.axis("off")

In [ ]:
# 10 imágenes más anómalas en test:

plt.figure(figsize=(12,12))
cont = 1
for ind in inds[100:110]:
  plt.subplot(10,10,cont)
  plot_image(X_te2[ind])
  cont = cont + 1
plt.show()
plt.figure(figsize=(12,12))
cont = 1
for ind in inds[100:110]:
  plt.subplot(10,10,cont)
  show_error(errores_signo[ind])
  cont = cont + 1